In [19]:
import java.io.File
import util.InputReader

typealias PuzzleInput = String

val exampleInput: PuzzleInput = InputReader.getExample(2023, 5)
val puzzleInput: PuzzleInput = InputReader.getPuzzleInput(2023, 5)

In [20]:

exampleInput

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4

In [21]:
fun PuzzleInput.seeds(): List<Long> = lines().first().split("seeds: ")[1].split(" ").map { it.toLong() }

exampleInput.seeds()

[79, 14, 55, 13]

In [22]:
data class SeedMapping(val from: String, val to: String, val sourceRange: LongRange, val offset: Long) {
    val destinationStart = sourceRange.first + offset
}

fun SeedMapping(line: String): SeedMapping {
    val (from, to, sourceBegin, sourceEnd, offset) = line.split(" ")
    return SeedMapping(from, to, sourceBegin.toLong()..sourceEnd.toLong(), offset.toLong())
} 

fun SeedMapping(from: String, to: String, string: String): SeedMapping {
    val (destinationBegin, sourceBegin, length) = string.split(" ").map { it.toLong() }
    val offset = destinationBegin - sourceBegin
    val sourceRange = sourceBegin..(sourceBegin + length)
    return SeedMapping(from, to, sourceRange, offset)
}

fun PuzzleInput.seedMappers() =
    lines().drop(2).joinToString("\n").split("\n\n")
        .flatMap { chunk ->
            val (header, rest) = chunk.split("\n", limit = 2)
            val (from, _, to) = header.split(" ")[0].split("-")
            rest.lines().map { SeedMapping(from, to, it) }
        }.groupBy { it.from }


exampleInput.seedMappers()

{seed=[SeedMapping(from=seed, to=soil, sourceRange=98..100, offset=-48), SeedMapping(from=seed, to=soil, sourceRange=50..98, offset=2)], soil=[SeedMapping(from=soil, to=fertilizer, sourceRange=15..52, offset=-15), SeedMapping(from=soil, to=fertilizer, sourceRange=52..54, offset=-15), SeedMapping(from=soil, to=fertilizer, sourceRange=0..15, offset=39)], fertilizer=[SeedMapping(from=fertilizer, to=water, sourceRange=53..61, offset=-4), SeedMapping(from=fertilizer, to=water, sourceRange=11..53, offset=-11), SeedMapping(from=fertilizer, to=water, sourceRange=0..7, offset=42), SeedMapping(from=fertilizer, to=water, sourceRange=7..11, offset=50)], water=[SeedMapping(from=water, to=light, sourceRange=18..25, offset=70), SeedMapping(from=water, to=light, sourceRange=25..95, offset=-7)], light=[SeedMapping(from=light, to=temperature, sourceRange=77..100, offset=-32), SeedMapping(from=light, to=temperature, sourceRange=45..64, offset=36), SeedMapping(from=light, to=temperature, sourceRange=64..7

In [23]:
typealias SeedMap = Map<LongRange, Long>
fun SeedMap(range: LongRange) = SeedMap(range.first, range.last)
fun SeedMap(min: Long, max: Long) = SeedMap(min, max, 0L)
fun SeedMap(range: LongRange, offset: Long) = SeedMap(range.first, range.last, offset)
fun SeedMap() = SeedMap(0L..Long.MAX_VALUE)

fun SeedMap(min: Long, max: Long, offset: Long): SeedMap = mapOf(min..max to offset)


SeedMap(0L..10L)
 

{0..10=0}

In [24]:
// split a range by another range
fun LongRange.split(by: LongRange): List<LongRange> {
    val points = listOf(first, last, by.first, by.last).sorted()
    return points.zipWithNext().map { it.first..it.second }
}
fun LongRange.overlaps(other: LongRange): Boolean = first <= other.last && other.first <= last
fun LongRange.contains(other: LongRange): Boolean = first <= other.first && other.last <= last

listOf(
    // should be [0..10, 10..20, 20..99]
    (0L..99L).split(10L..20L),
    (0L..10L).split(80L..90L)
)

[[0..10, 10..20, 20..99], [0..10, 10..80, 80..90]]

In [25]:
fun SeedMap.splitBy(range: LongRange) = flatMap { (key, value) ->
    if (key.overlaps(range)) {
        key.split(range).map { it to value }
    } else {
        listOf(key to value)
    }
}.toMap()

SeedMap(0L, 99L).splitBy(10L..20L).splitBy(5L..15L)

{0..5=0, 5..10=0, 10..15=0, 15..20=0, 20..99=0}

In [26]:
fun SeedMap.splitBy(range: LongRange, newOffset: Long): SeedMap {
    val split = splitBy(range).toMutableMap()
    for (key in split.keys) {
        if (range.contains(key)) {
            split[key] = newOffset
        }
    }
    return split
}

SeedMap(0L..30L).splitBy(10L..20L, 5L).splitBy(5L..15L, 10L)

{0..5=0, 5..10=10, 10..15=10, 15..20=5, 20..30=0}

In [34]:
fun SeedMap.splitBy(mapper: SeedMapping) = splitBy(mapper.sourceRange, mapper.offset)
SeedMap(0L, 100L).splitBy(exampleInput.seedMappers()["soil"]!!.first())

{0..15=0, 15..52=-15, 52..100=0}

In [35]:
fun SeedMap.assertBounds(seed: Long) {
    val minOf = this.keys.minOf { it.first }
    if (seed < minOf || seed > this.keys.maxOf { it.last }) {
        throw Exception("Seed $seed is out of bounds")
    }
}

fun SeedMap.mapSeed(seed: Long): Long {
    this.assertBounds(seed)
    return this.entries.first { it.key.contains(seed) }.value + seed
}

SeedMap(10L, 20L, 2).mapSeed(10L)

12

In [36]:
fun SeedMapping.mapSeed(seed: Long) = if (sourceRange.contains(seed)) offset + seed else seed
listOf(97L, 98L, 99L, 100L, 101L).map { exampleInput.seedMappers()["soil"]!!.first().mapSeed(it) }

[97, 98, 99, 100, 101]

In [ ]:
fun partOne(input: PuzzleInput): Long {}